In [1]:
# Import necessary libraries
from openprompt.plms import T5TokenizerWrapper
from datasets import load_from_disk
from openprompt.pipeline_base import PromptDataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
from openprompt.prompts import ManualTemplate, MixedTemplate
from openprompt import PromptForClassification
from openprompt.data_utils import FewShotSampler
from random import shuffle
from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup
import torch
from openprompt.prompts import ManualVerbalizer
from openprompt.data_utils import InputExample
from tqdm import tqdm
import json

dataset_path = "/lustre/work/client/users/minhos/cache/datasets/p3_ropes_rc"
raw_dataset = load_from_disk(dataset_path)


dataset = {}
for split in ['train', 'validation']:
    dataset[split] = []
    raw_dataset[split] = raw_dataset[split].select(range(500))
    for idx, data in enumerate(raw_dataset[split]):
        dataset[split].append(data)

print(dataset['train'][0])
print(type(dataset['train'][0]))


# Load the T5 model
from openprompt.plms import load_plm
t5_path = "/lustre/work/client/users/minhos/models_for_supercomputer/t5-base"
model = T5ForConditionalGeneration.from_pretrained(t5_path)
tokenizer = T5Tokenizer.from_pretrained(t5_path)




# Logging setup
log_file = "qa_manual_template_multi_binary_t5.json"
results = []

# For Training, select 18 samples with the same context and different questions
data1 = dataset['train'][2:20]
dataset1 = []
for idx, data in enumerate(data1):
    question = data["inputs_pretokenized"]
    correct_answer = data["targets_pretokenized"].strip()
    label = 0 if correct_answer in ['cell X', 'Cell A', 'larger', 'more'] else 1  
    # Create an InputExample
    input_example = InputExample(
        text_a=question,
        label=label,
        guid=idx, # Assign a dummy label since there is only one answer
        meta={"correct_answer": correct_answer}
    )
    dataset1.append(input_example)

template1 = ManualTemplate(
    tokenizer=tokenizer,
    text='{"placeholder":"text_a"} The answer is: {"mask"}',
)
verbalizer1 = ManualVerbalizer(
    tokenizer=tokenizer,
    num_classes=2,
    label_words=[['cell X','cell A', 'larger','more'],['cell Z', 'cell B', 'smaller', 'less']]
)

prompt_model = PromptForClassification(
    plm=model,
    template=template1,
    verbalizer=verbalizer1,
    freeze_plm=False,
)
train_dataloader = PromptDataLoader(
    dataset = dataset1,
    template=template1,
    tokenizer=tokenizer,
    tokenizer_wrapper_class=T5TokenizerWrapper,
    decoder_max_length=68, max_seq_length=480,
    batch_size=1)

loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
from tqdm import tqdm
optimizer = AdamW(optimizer_grouped_parameters, lr=0.0001)
prompt_model.train()
for epoch in range(10):
    tot_loss = 0
    pbar = tqdm(train_dataloader, desc="Training")
    for step, inputs in enumerate(train_dataloader):
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        pbar.set_postfix({"loss": tot_loss / (step + 1)})
        if step %100 ==1:
            print("Epoch {}, average loss: {}".format(epoch+1, tot_loss/(step+1)), flush=True)


data2 = dataset['validation'][:1000]
dataset2 = []
for idx, data in enumerate(data2):
    question = data["inputs_pretokenized"]
    correct_answer = data["targets_pretokenized"].strip()
    # Create an InputExample
    input_example = InputExample(
        text_a=question,
        label= 0,
        guid=idx, # Assign a dummy label since there is only one answer
        meta={"correct_answer": correct_answer}
    )
    dataset2.append(input_example)
    
for idx, data in enumerate(dataset2):

    template2 = ManualTemplate(
        tokenizer=tokenizer,
        text='{"placeholder":"text_a"} The answer is: {"mask"}',
    )
    
    verbalizer2 = ManualVerbalizer(
        tokenizer=tokenizer,
        num_classes=2,
        label_words=[[data.meta['correct_answer']],["other"]]
    )

    prompt_model.template = template2
    prompt_model.verbalizer = verbalizer2

    validation_dataloader = PromptDataLoader(
        dataset = [data],
        template=template2,
        tokenizer=tokenizer,
        tokenizer_wrapper_class=T5TokenizerWrapper,
        decoder_max_length=3, max_seq_length=480,
        batch_size=1
        )

    prompt_model.eval()
    with torch.no_grad():
        for inputs in validation_dataloader:
            logits = prompt_model(inputs)
            preds = torch.argmax(logits, dim=-1)
            correct = preds.item() == data.label
    
    results.append({"index": idx, "correct": correct})

# Compute overall accuracy
accuracy = sum(r["correct"] for r in results) / len(results)
print(f"Validation Accuracy: {accuracy:.4f}")

# Save results to JSON
with open(log_file, "w") as f:
    json.dump(results, f, indent=4)

/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


{'inputs': [27, 54, 169, 48, 2458, 10, 3424, 757, 1855, 6986, 116, 3, 9, 8096, 9016, 190, 8, 2358, 13304, 406, 174, 53, 136, 827, 12, 1903, 190, 5, 100, 2906, 116, 3, 9, 8096, 6914, 45, 46, 616, 213, 34, 19, 72, 18054, 12, 46, 616, 213, 34, 19, 705, 18054, 5, 27664, 257, 19, 8, 381, 13, 14219, 13, 3, 9, 8096, 16, 3, 9, 787, 2908, 5, 1563, 31, 7, 497, 25, 27157, 3, 9, 21776, 13, 3136, 16, 3, 9, 4119, 13, 387, 5, 37, 29, 25, 27157, 192, 21776, 7, 13, 3136, 16, 430, 4119, 13, 387, 5, 37, 511, 1127, 56, 43, 3, 9, 1146, 6145, 13, 3136, 5, 852, 6, 27, 43, 3, 9, 126, 1419, 10, 71, 388, 474, 192, 12294, 6, 4119, 71, 11, 4119, 272, 6, 3353, 28, 4081, 6201, 13, 387, 30, 12, 3, 9, 953, 11, 3, 6412, 550, 12, 281, 691, 112, 4842, 5, 978, 520, 764, 590, 11, 1509, 8, 192, 12294, 11, 1500, 12, 474, 128, 2656, 16, 135, 12, 143, 3, 9, 11915, 3281, 5, 37, 861, 3, 22929, 192, 14987, 1329, 7, 13, 2656, 139, 4119, 71, 11, 386, 14987, 1329, 7, 13, 2656, 139, 4119, 272, 5, 11801, 48, 822, 754, 10, 4073, 4119,

/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

Epoch 1, average loss: 0.9402022957801819


Training:   0%|          | 0/18 [01:21<?, ?it/s, loss=0.601]

Training:   0%|          | 0/18 [00:04<?, ?it/s, loss=0.847]

...

Epoch 9, average loss: 0.09022496757097542


Training:   0%|          | 0/18 [00:41<?, ?it/s, loss=0.129]

Training:   0%|          | 0/18 [00:04<?, ?it/s, loss=0.0659]

Epoch 10, average loss: 0.0658742911182344



Training:   0%|          | 0/18 [00:40<?, ?it/s, loss=0.0648]
tokenizing: 1it [00:00, 103.59it/s]:43<?, ?it/s, loss=0.0669]
tokenizing: 1it [00:00, 275.36it/s]
tokenizing: 1it [00:00, 167.85it/s]
tokenizing: 1it [00:00, 233.55it/s]
tokenizing: 1it [00:00, 132.14it/s]
tokenizing: 1it [00:00, 173.92it/s]
tokenizing: 1it [00:00, 172.89it/s]
tokenizing: 1it [00:00, 166.88it/s]
tokenizing: 1it [00:00, 163.62it/s]
tokenizing: 1it [00:00, 190.32it/s]
tokenizing: 1it [00:00, 138.56it/s]
tokenizing: 1it [00:00, 176.68it/s]
tokenizing: 1it [00:00, 260.10it/s]
tokenizing: 1it [00:00, 180.82it/s]
tokenizing: 1it [00:00, 257.97it/s]
tokenizing: 1it [00:00, 258.03it/s]
tokenizing: 1it [00:00, 253.86it/s]
tokenizing: 1it [00:00, 167.16it/s]
tokenizing: 1it [00:00, 501.29it/s]
tokenizing: 1it [00:00, 250.05it/s]
tokenizing: 1it [00:00, 239.31it/s]
tokenizing: 1it [00:00, 254.66it/s]
tokenizing: 1it [00:00, 245.86it/s]
tokenizing: 1it [00:00, 482.83it/s]
tokenizing: 1it [00:00, 119.87it/s]
tokenizing:

Validation Accuracy: 0.8820
